In [172]:
import numpy as np
import pandas as pd
import sys
import pickle as pkl

import pandas as pd
import pickle as pkl
import sys

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

import numpy as np
import os
from matplotlib import pyplot as plt

from scipy.special import expit as sigmoid
from PIL import Image

In [ ]:
header_data = '../data/'

## extract gender age device from CT dicoms

In [86]:
df = pd.read_csv(os.path.join(header_data, 'data.csv'))
#data file has CT_acc (CT accessiom number), 'TOTAL_SPINE_TSCORE', 'Original DICOM file location-Axial', 'Original DICOM file location-Coronal'
len(df), 'TOTAL_SPINE_TSCORE' in df.columns

(2884, True)

In [188]:
meta = pd.concat(os.path.join(header_data, 'niftis/metadata.csv'))
mapp = pd.concat(os.path.join(header_data, 'niftis/mapping.csv'))
len(meta), len(mapp)

Columns (138,139,180) have mixed types.Specify dtype option on import or set low_memory=False.
Columns (35,178,198,199,200,202) have mixed types.Specify dtype option on import or set low_memory=False.


(23072, 12888)

In [88]:
meta_mapped = meta.loc[meta.file.isin(mapp['Original DICOM file location'].values)]
len(meta_mapped)

12888

In [93]:
cols = [ 'Manufacturer', 'ManufacturerModelName', 'OtherPatientIDs', 'PatientAge', 
        'PatientBirthDate', 'PatientID', 'PatientSex', 'PatientSize', 'PatientWeight', 
        'ImageOrientationPatient', 'ImagePositionPatient', 'PatientPosition', 
        'PatientOrientation',  'PixelSpacing_Axial',  'PixelSpacing_Coronal']
axial  = 0
coronal = 0
df[cols] = None
for i,j in df.iterrows():
    temp = meta_mapped.loc[(meta_mapped.AccessionNumber==df.at[i, 'CT_acc']) & (meta_mapped.file==df.at[i, 'Original DICOM file location-Transverse'])]
    if len(temp)>0:
        df.at[i, 'PixelSpacing_Axial'] = temp.at[temp.index[0], 'PixelSpacing']
        axial +=1
    else:
        temp = meta_mapped.loc[(meta_mapped.AccessionNumber==df.at[i, 'CT_acc']) & (meta_mapped.file==df.at[i, 'Original DICOM file location-Coronal'])]
        if len(temp)>0:
            df.at[i, 'PixelSpacing_Coronal'] = temp.at[temp.index[0], 'PixelSpacing']
            coronal+=1
    if len(temp)>0:
        for c in cols:
            df.at[i, c] = temp.at[temp.index[0], c]
    if i%100==0:
        print(axial, coronal,  i)
print(axial, coronal, i)

2884
1 0 0 0 0
101 0 0 0 100
201 0 0 0 200
301 0 0 0 300
401 0 0 0 400
501 0 0 0 500
601 0 0 0 600
701 0 0 0 700
801 0 0 0 800
901 0 0 0 900
1001 0 0 0 1000
1101 0 0 0 1100
1201 0 0 0 1200
1301 0 0 0 1300
1401 0 0 0 1400
1501 0 0 0 1500
1601 0 0 0 1600
1701 0 0 0 1700
1801 0 0 0 1800
1901 0 0 0 1900
2001 0 0 0 2000
2101 0 0 0 2100
2201 0 0 0 2200
2301 0 0 0 2300
2401 0 0 0 2400
2501 0 0 0 2500
2601 0 0 0 2600
2701 0 0 0 2700
2801 0 0 0 2800
2884 0 0 0 2883


In [95]:
df.to_csv(os.path.join(header_data, 'data_w_ehr_info.csv'))

In [96]:
for c in cols:
    print(c, end='\t\t\t')
    print(np.round(len(df.dropna(subset=[c]))/len(df), decimals=4))

Manufacturer			1.0
ManufacturerModelName			1.0
OtherPatientIDs			0.1252
PatientAge			0.932
PatientBirthDate			1.0
PatientID			1.0
PatientSex			1.0
PatientSize			0.1141
PatientWeight			0.1221
ImageOrientationPatient			1.0
ImagePositionPatient			1.0
PatientPosition			1.0
PatientOrientation			0.051


### Cross Section

In [ ]:
import skimage
from scipy import ndimage
df = pd.read_csv(os.path.join(header_data, 'data_w_ehr_info.csv'))
def vol_window(vol, level, window):
    maxval = level + window/2
    minval = level - window/2
    vol[vol<minval] = minval
    vol[vol>maxval] = maxval
    return vol
f = plt.figure(figsize=(16,16))
for ii in range(0, len(df)):
    
    i = df.index[ii]
    CT_acc = df.at[i, 'CT_acc']
    slice_number = int(df.at[i, 'L3_slice'])

    header = os.path.join(header_data, 'slices/axial/')
    image_name = header+str(CT_acc)+'_'+str(slice_number)+'.npy'
    a = np.load(image_name)
    a = a.astype(float)

    a = vol_window(a, 500, 1500)
    a = (a-np.min(a))/(np.max(a)-np.min(a))
    a = 255.0*a
    mask = a.copy()
    mask[mask>0] = 255
    image = Image.fromarray(np.uint8(a)).convert('RGB')
    mask_image = Image.fromarray(np.uint8(mask)).convert('RGB')

    
    labels = skimage.measure.label(mask, return_num=False)

    maxCC_withbcg = labels == np.argmax(np.bincount(labels.flat))
    maxCC_nobcg = labels == np.argmax(np.bincount(labels.flat, weights=mask.flat))
    largest = maxCC_nobcg.copy()
    largest[largest==False] = 0
    largest[largest==True] = 1
    
    largest_filled = ndimage.binary_fill_holes(largest).astype(int)
    max_width = max([len([i for i in range(512) if largest_filled[i, y]>0]) for y in range(512)])
    max_depth = max([len([i for i in range(512) if largest_filled[y,i]>0]) for y in range(512)])
    
    #print(max_width, max_depth)
    df.at[i, 'Width'] = max_width
    df.at[i, 'Depth'] = max_depth

    if ii % 100==0:
        print(ii)

df['CrossSection'] = np.sqrt(df['Width']*df['Depth'])

def pixelspacing_0(x):
    return float(x.split(',')[0][2:-1])
def pixelspacing_1(x):
    return float(x.split(',')[1][2:-2])
df['PixelSpacing_Transverse_0'] = df.PixelSpacing_Transverse.apply(pixelspacing_0)
df['PixelSpacing_Transverse_1'] = df.PixelSpacing_Transverse.apply(pixelspacing_1)
a = df['PixelSpacing_Transverse_1'] == df['PixelSpacing_Transverse_0']

df['WidthSpacing'] = df['Width']*df['PixelSpacing_Transverse_1']
df['DepthSpacing'] = df['Depth']*df['PixelSpacing_Transverse_0']
a = df['WidthSpacing']*df['DepthSpacing']
df['CrossSectionSpacing'] = np.sqrt(a)

df.to_csv(os.path.join(header_data, 'data_w_ehr_info.csv'))

### Implants

In [ ]:
from PIL import Image
def vol_window(vol, level, window):
    maxval = level + window/2
    minval = level - window/2
    vol[vol<minval] = minval
    vol[vol>maxval] = maxval
    return vol
df = pd.read_csv(os.path.join(header_data, 'data_w_ehr_info.csv'))

for index in range(0, len(df)):
    header = os.path.join(header_data, 'slices/axial/')
    y = df.at[df.index[index], 'TOTAL_SPINE_TSCORE']

    CT_acc = df.at[df.index[index], 'CT_acc']
    slice_number = int(df.at[df.index[index], 'L3_slice'])

    image_name = header+str(CT_acc)+'_'+str(slice_number)+'.npy'
    a = np.load(image_name)
    a = a.astype(float)
    bins = [-1024, 1500, 2500, 5000]
    hist, bin_edges = np.histogram(a, bins = bins)
    df.at[df.index[index], '-1024_1500'] = hist[0]
    df.at[df.index[index], '1500_2500'] = hist[1]
    df.at[df.index[index], '2500_5000'] = hist[2]
    if hist[1]>250 and hist[2]>250:
        df.at[df.index[index], 'Implant'] = True
    else:
        df.at[df.index[index], 'Implant'] = False
    if index%100==0:
        print(index)
df.to_csv(os.path.join(header_data, 'data_w_ehr_info.csv'))

### MRN (Patient ID) based split

In [44]:
import pickle as pkl 
mrn = list(df.MRN.values)
mrn_train = np.random.choice(MRN, round(0.8*len(mrn)), replace=False)
mrn_test = [i for i in mrn if i not in mrn_train]
mrn_val = np.random.choice(mrn_train, round(0.1*len(mrn_train)), replace=False)
MRN_train = [i for i in mrn_train if i not in mrn_val]


dct = {'mrn_train':mrn_train, 'mrn_test': mrn_test, 'mrn_val': mrn_val}
pkl.dump(dct, open(os.path.join(header_data, 'mrn_split.pkl'), 'wb'))

## Baseline Model, Age and Gender + Effective Diameter

In [4]:
import pandas as pd
import pickle as pkl
import sys

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn.neural_network import MLPClassifier




import numpy as np
import os
from matplotlib import pyplot as plt

In [46]:
def labeling(x):
    if x<=-1:
        return 1
    else:
        return 0
def age_mapping(x):
    if type(x) is str and len(x)==4:
        return x[1]
    elif type(x) is str and len(x)==3:
        return x[0]
    else:
        return 'UNKNOWN'

#136.1302176492294, 462.9853871884079
df = pd.read_csv(os.path.join(header_data, 'data_w_ehr_info.csv'))
print(len(df))
df = df.dropna(subset=['Original DICOM file location-Axial', 'Original DICOM file location-Coronal', 'TOTAL_SPINE_TSCORE', 'L3_slice'], how='any')
print(len(df))

df['Label'] = df['TOTAL_SPINE_TSCORE'].apply(labeling)
print(df['Label'].value_counts())

df['PatientAge_mapped'] = df.PatientAge.apply(age_mapping)
print(df['PatientAge_mapped'].value_counts())


df['CrossSectionSpacing_binned'] = pd.cut(df['CrossSectionSpacing'].values, 10)

a = df['CrossSectionSpacing'].values
df['CrossSectionSpacing_norm'] = (a-np.min(a))/(np.max(a)-np.min(a))



dct = pkl.load(open(os.path.join(header_data, 'mrn_split'), 'rb'))
mrn_train = dct['mrn_train']
mrn_test = dct['mrn_test']
mrn_val = dct['mrn_val']

df_train = df.loc[df.mrn.isin(mrn_train)]
df_test = df.loc[df.mrn.isin(mrn_test)]
df_val = df.loc[df.mrn.isin(mrn_val)]

print(df_train['Label'].value_counts())
print(df_test['Label'].value_counts())
print(df_val['Label'].value_counts())

labels_train = df_train.Label.values
labels_test = df_test.Label.values
labels_val = df_val.Label.values

female_idx = np.array([i for i in range(len(df_test)) if df_test.PatientSex.values[i]=='F'])
male_idx = np.array([i for i in range(len(df_test)) if df_test.PatientSex.values[i]=='M'])

6083
6083
0    3108
1    2975
Name: Label, dtype: int64
6          2385
7          1482
5          1384
8           555
UNKNOWN     222
9            55
Name: PatientAge_mapped, dtype: int64
0    2209
1    2199
Name: Label, dtype: int64
0    670
1    535
Name: Label, dtype: int64
1    241
0    229
Name: Label, dtype: int64


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (18,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
discrete_cols = ['PatientAge_mapped', 'PatientSex', 'CrossSectionSpacing_binned']

npd_discrete = df_train[discrete_cols].copy().to_numpy()
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(npd_discrete)


mat = enc.transform(npd_discrete).todense()

npd_discrete = df_test[discrete_cols].copy().to_numpy()


mat_test = enc.transform(npd_discrete).todense() 

print(mat.shape, mat_test.shape)

mat = np.concatenate((mat[:, :5], mat[:, 6:]), axis=1)
mat_test = np.concatenate((mat_test[:, :5], mat_test[:, 6:]), axis=1)
print(mat.shape, mat_test.shape)

clf = MLPClassifier(random_state=0)
clf.fit(mat, labels_train)
pkl.dump(clf, open(os.path.join('Models/EHR/model.sav'), 'wb'))
##########################
preds = clf.predict(mat)
probs = clf.predict_proba(mat)
dct = {}
dct['labels'] = labels_train
dct['preds'] = preds
dct['probs'] = probs
pkl.dump(dct, open(os.path.join('Models/EHR/train_predictions.pkl'), 'wb'))
#######################
preds = clf.predict(mat_test)
probs = clf.predict_proba(mat_test)

print(classification_report(labels_test, preds))
print('AUC-ROC:\t', roc_auc_score(labels_test, probs[:,1]))
cm = confusion_matrix(labels_test, preds, labels = [0, 1])
print('Confusion Matrix:\n', cm)
dct = {}
dct['labels'] = labels_test
dct['preds'] = preds
dct['probs'] = probs
pkl.dump(dct, open(os.path.join('Models/EHR/test_predictions.pkl'), 'wb'))


(4408, 18) (1205, 18)
(4408, 17) (1205, 17)
              precision    recall  f1-score   support

           0       0.71      0.69      0.70       670
           1       0.63      0.65      0.64       535

    accuracy                           0.68      1205
   macro avg       0.67      0.67      0.67      1205
weighted avg       0.68      0.68      0.68      1205

AUC-ROC:	 0.7059045892035152
Confusion Matrix:
 [[464 206]
 [185 350]]


shap_values[1].shape

In [17]:
for gender in ['F', 'M']:
    print(gender)
    gender_idx = np.array([i for i in range(len(df_test)) if df_test.PatientSex.values[i]==gender])
    print(classification_report(labels_test[gender_idx], preds[gender_idx]))
    print('AUC-ROC:\t', roc_auc_score(labels_test[gender_idx], probs[gender_idx,1]))
    cm = confusion_matrix(labels_test[gender_idx], preds[gender_idx], labels = [0, 1])
    print('Confusion Matrix:\n', cm)


FEMALE
              precision    recall  f1-score   support

           0       0.63      0.49      0.55       359
           1       0.64      0.76      0.70       426

    accuracy                           0.64       785
   macro avg       0.64      0.63      0.62       785
weighted avg       0.64      0.64      0.63       785

AUC-ROC:	 0.6561000170007978
Confusion Matrix:
 [[175 184]
 [101 325]]
MALE
              precision    recall  f1-score   support

           0       0.77      0.93      0.85       311
           1       0.53      0.23      0.32       109

    accuracy                           0.75       420
   macro avg       0.65      0.58      0.58       420
weighted avg       0.71      0.75      0.71       420

AUC-ROC:	 0.5934835835865366
Confusion Matrix:
 [[289  22]
 [ 84  25]]


In [18]:
companies = ['ge', 'siemens', 'toshiba']
for c in companies:
    print(c.upper())
    idx = np.array([i for i in range(len(df_test)) if c in df_test.Manufacturer.values[i].lower()])
    #print(idx)
    print(classification_report(labels_test[idx], preds[idx]))
    print('AUC-ROC:\t', roc_auc_score(labels_test[idx], probs[idx,1]))
    cm = confusion_matrix(labels_test[idx], preds[idx], labels = [0, 1])
    print('Confusion Matrix:\n', cm)

GE
              precision    recall  f1-score   support

           0       0.71      0.76      0.74        85
           1       0.69      0.63      0.66        70

    accuracy                           0.70       155
   macro avg       0.70      0.70      0.70       155
weighted avg       0.70      0.70      0.70       155

AUC-ROC:	 0.7653781512605042
Confusion Matrix:
 [[65 20]
 [26 44]]
SIEMENS
              precision    recall  f1-score   support

           0       0.72      0.68      0.69       561
           1       0.62      0.66      0.64       442

    accuracy                           0.67      1003
   macro avg       0.67      0.67      0.67      1003
weighted avg       0.67      0.67      0.67      1003

AUC-ROC:	 0.6964797025350659
Confusion Matrix:
 [[379 182]
 [151 291]]
TOSHIBA
              precision    recall  f1-score   support

           0       0.71      0.83      0.77        24
           1       0.79      0.65      0.71        23

    accuracy             

In [19]:
dia = [0, 250, 350, 1000]
for i in range(1, len(dia)):
    lower = dia[i-1]
    upper = dia[i]
    print(str(lower)+'-'+str(upper))
    idx = np.array([i for i in range(len(df_test)) if (df_test.CrossSectionSpacing.values[i]>=lower and df_test.CrossSectionSpacing.values[i]<upper)])
    #print(idx)
    print(classification_report(labels_test[idx], preds[idx]))
    print('AUC-ROC:\t', roc_auc_score(labels_test[idx], probs[idx,1]))
    cm = confusion_matrix(labels_test[idx], preds[idx], labels = [0, 1])
    print('Confusion Matrix:\n', cm)

0-250
              precision    recall  f1-score   support

           0       1.00      0.05      0.10        56
           1       0.70      1.00      0.82       123

    accuracy                           0.70       179
   macro avg       0.85      0.53      0.46       179
weighted avg       0.79      0.70      0.60       179

AUC-ROC:	 0.5643873403019745
Confusion Matrix:
 [[  3  53]
 [  0 123]]
250-350
              precision    recall  f1-score   support

           0       0.68      0.68      0.68       472
           1       0.60      0.60      0.60       380

    accuracy                           0.64       852
   macro avg       0.64      0.64      0.64       852
weighted avg       0.64      0.64      0.64       852

AUC-ROC:	 0.6552157671721678
Confusion Matrix:
 [[319 153]
 [153 227]]
350-1000
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       142
           1       0.00      0.00      0.00        32

    accuracy     

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [20]:
##confidence interval
from random import randint, sample


dct = pkl.load(open(os.path.join('Models/ehr_test_predictions.pkl'), 'rb'))
y_test = dct['labels']
preds = dct['preds']
probs = dct['probs']


target_test = y_test

avg_precision = []
avg_recall = []
avg_fscore = []
aucroc = []
   
test_set_size = len(target_test)
for i in range(1000):
    # randomly pick size of the test set
    i_size = randint(round(0.5*test_set_size), test_set_size)
    
    i_test_idx = sample([ii for ii in range(test_set_size)], i_size)
    i_test_idx.sort()
    
    


    i_y_test = target_test[i_test_idx]
    i_y_pred = preds[i_test_idx]
    i_y_prob = probs[i_test_idx,1]

    dct = classification_report(i_y_test, i_y_pred, output_dict=True, zero_division=0)
    avg_precision.append(dct['macro avg']['precision'])
    avg_recall.append(dct['macro avg']['recall'])
    avg_fscore.append(dct['macro avg']['f1-score'])

    aucroc.append(roc_auc_score(i_y_test, i_y_prob))
    if i%100==0:
        print('Iteration:\t'+str(i))
        
# confidence intervals
alpha = 0.95
print('%.1f confidence interval ' % (alpha*100))


p = ((1.0-alpha)/2.0) * 100
lower = np.percentile(avg_precision, p, axis= 0)
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = np.percentile(avg_precision, p, axis = 0)
print('Precision')
print('['+str(np.round(lower*100, decimals=1))+'-'+str(np.round(upper*100, decimals=1))+']')

p = ((1.0-alpha)/2.0) * 100
lower = np.percentile(avg_recall, p, axis= 0)
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = np.percentile(avg_recall, p, axis = 0)
print('Recall')
print('['+str(np.round(lower*100, decimals=1))+'-'+str(np.round(upper*100, decimals=1))+']')

p = ((1.0-alpha)/2.0) * 100
lower = np.percentile(avg_fscore, p, axis= 0)
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = np.percentile(avg_fscore, p, axis = 0)
print('F-score')
print('['+str(np.round(lower*100, decimals=1))+'-'+str(np.round(upper*100, decimals=1))+']')

p = ((1.0-alpha)/2.0) * 100
lower = np.percentile(aucroc, p, axis= 0)
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = np.percentile(aucroc, p, axis = 0)
print('AUC-ROC')
print('['+str(np.round(lower*100, decimals=1))+'-'+str(np.round(upper*100, decimals=1))+']')

Iteration:	0
Iteration:	100
Iteration:	200
Iteration:	300
Iteration:	400
Iteration:	500
Iteration:	600
Iteration:	700
Iteration:	800
Iteration:	900
95.0 confidence interval 
Precision
[65.5-69.1]
Recall
[65.6-69.2]
F-score
[65.5-69.1]
AUC-ROC
[68.6-72.6]
